In [76]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

headers = {'content-type': 'text/html; charset=UTF-8','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}

driver = webdriver.Chrome()
driver.get("https://www.money101.com.tw/%E4%BF%A1%E7%94%A8%E5%8D%A1/%E7%B2%BE%E9%81%B8%E6%8E%A8%E8%96%A6")

time.sleep(5) #暫停

#soup = BeautifulSoup(driver.page_source, 'html.parser')
#print(soup.prettify())  #輸出排版後的HTML內容

button= driver.find_element(By.XPATH, "/html/body/div[1]/div/nav/div[1]/div[2]/ul/div[3]/li/label") #
if button.is_displayed() and button.is_enabled():
    button.click()
#button.click()
time.sleep(2)
        
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀

result = soup.select('html body div:nth-of-type(1) div nav div:nth-of-type(1) div:nth-of-type(2) ul div:nth-of-type(3) li div div')
#print(result)
ranking_link = []
ranking_title = []
for item in result:
    link = item.find('a')
    if link:
        ranking_link.extend(['https://www.money101.com.tw' + a.get('href') for a in item.find_all('a')])
    span = item.find('span')
    if span:
        rank = span.text
        ranking_title.append(rank)
        
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)
ranking_title.pop(0)

ranking_link.pop(0)
ranking_link.pop(0)
ranking_link.pop(0)

#print(ranking_link)
#print(ranking_title)
#print(len(ranking_link))
#print(len(ranking_title))


for k in range(len(ranking_link)-1):
    
    driver = webdriver.Chrome()
    driver.get(ranking_link[k+1])
    time.sleep(5) #暫停
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #print(soup.prettify())  #輸出排版後的HTML內容
    
    
    card_name = []
    card_link = []
    cards = soup.find_all('div',class_="flex items-center justify-between")
    for card in cards:
        title = card.find('h2',class_='pr-1')
        link = card.find('a').get('href')
        card_name.append(title.text)
        print('☆'+title.text)
        print('https://www.money101.com.tw'+link+'\n')
        card_link.append('https://www.money101.com.tw'+link)
        
    #單卡
    #最低年齡要求
    age_request = []
    #最低年薪要求
    salary_request = []
    #國外刷卡手續費
    fee = []
    #年費
    annual_fee = []
    #循環信用利率
    rate = []
    #信用卡發卡機構
    agency = []
    #card_name = []
    card_list2 = ['最低年齡要求','最低年薪要求','國外刷卡手續費','年費','循環信用利率','信用卡發卡機構']
        
    for j in range(len(card_link)):
    
        driver.get(card_link[j]) 
        
        time.sleep(5) #暫停
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') #前往第一張卡的網頁
        
        print(card_name[j])
        
        result1 = soup.find_all('div', class_="box-border inline-block w-2/5 pr-2 align-top")#標題        
        card_list = []    #每張卡的標題（不一定是六個）    
        for result6 in result1:    
            card_list.append(result6.text)
            print(result6.text)
        
        result2 = soup.find_all('div', class_="inline-block w-1/2") #內文
        n=0
        for result in result2:
            if card_list[n] == "最低年齡要求":
                age_request.append(result.text) 
            elif card_list[n] == "最低年薪要求":
                salary_request.append(result.text)
            elif card_list[n] == "年費":
                fee.append(result.text)
            elif card_list[n] == "國外刷卡手續費":
                annual_fee.append(result.text)
            elif card_list[n] == "循環信用利率":
                rate.append(result.text)
            elif card_list[n] == "信用卡發卡機構":
                agency.append(result.text)       
            print(card_list[n]+":"+result.text+'\n')
            n+=1
            
        if len(age_request) != j+1:
            age_request.append('不適用')
        if len(salary_request) != j+1:
            salary_request.append('不適用')
        if len(fee) != j+1:
            fee.append('無')
        if len(annual_fee) != j+1:
            annual_fee.append('不適用')
        if len(rate) != j+1:
            rate.append('不適用')
        if len(agency) != j+1:
            agency.append('不適用')
            
        
        
        
        #table
        #可能沒有
        table = soup.find('div', class_='min-w-0 flex-1 md:ml-2')
        tables = table.find_all('table', style = 'width:100%') #如果有table
        title_list=[] #優惠csv
        text_list =[] #優惠csv
        if tables and ranking_title[k+1]== '台新銀行 台新街口聯名卡':
            title_list.append('活動一')
            title_list.append('活動二')
        #print(table)
        if tables:
            for table in tables:
                if table != None  and ranking_title[k+1]== '滙豐銀行 匯鑽卡':
                    results = table.find_all('tr')
                    i=0
                    header =[]
                    a = []
                    #b = []
                    for result in results:
                        r = result.find_all('td')
                        if i == 0:
                            for r in result:
                                #print(r.text)
                                header.append(r.text)
                            #print(i)
                        elif i ==1:
                            for r in result:
                                #print(r.text)
                                a.append(r.text)
                            #print(i)
                    #    elif i ==2:
                    #        for r in result:
                                #print(r.text)
                    #            b.append(r.text)
                            #print(i)
                        i=i+1
                    
                    for i in header:
                        if ' ' in header:
                            header.remove(' ')
                            
                    for i in a:
                        if ' ' in a:
                            a.remove(' ')
                            
                    #for i in b:
                    #    if ' ' in b:
                    #        b.remove(' ')
                    #b[0] = b[0][0:4]
                    print(header)
                    print(a)
                    #print(b)
                    #d = [a]
                    #df = pd.DataFrame(d, columns = header)
                    #print(df)
                    title_list.append(header[1]+'的'+a[0])
                    title_list.append(header[2]+'的'+a[0])
                    text_list.append(a[1])
                    text_list.append(a[2])
                    print(header[1]+'的'+a[0]+': '+a[1])
                    print(header[2]+'的'+a[0]+': '+a[2])
                #   print(header[1]+'的'+b[0]+': '+b[1])
                #   print(header[2]+'的'+b[0]+': '+b[2])
                elif table != None  and ranking_title[k+1]== '台新銀行 台新街口聯名卡':
                    table_header = []
                    table_text = []
                    trs = table.find_all('tr')
                    for tr in trs:
                        ths = tr.find_all('th')
                        tds = tr.find_all('td')
                        if ths !=None:
                            i = 0
                            for th in ths:
                                if i == 0:
                                    table_header.append(th.text)
                                elif i == 1:
                                    table_text.append(th.text)
                                print(th.text)
                                i+=1
                        else:
                            i = 0
                            for th in ths:
                                if i == 0:
                                    table_header.append(td.text)
                                elif i == 1:
                                    table_text.append(td.text)
                                print(td.text)
                                i+=1
                    texts=[]
                    for i in  range(len(table_header)):
                        texts.append(table_header[i]+':'+table_text[i])
                    t = '\n'.join(texts)
                    text_list.append(t)
                
                

        #聯邦銀行 吉鶴卡
        results = soup.find_all('div', class_='min-w-0 flex-1 md:ml-2') #產品特色、注意事項
        for result in results:
            print(result.find('p').text) #”產品特色“ “注意事項”
            result1 = result.find('ul', class_='-my-1') #產品特色大框、注意事項大框
            result2 = result1.find_all('li', class_='my-1 flex gap-1 font-small') #產品特色 勾 注意事項 橫槓
            for re in result2:
                text1 = re.find('div', class_ = 'rich-text w-11/12 md:w-full')
                table = text1.find('table', style = 'width:100%') #如果有table
                if table != None:
                    result3 = text1.find_all('li')
                    for res in result3:
                        print(res.text+'\n')
                        text_list.append(res.text)
                        title_list.append(result.find('p').text)
                else:
                    print(text1.text+'\n')
                    text_list.append(text1.text)
                    title_list.append(result.find('p').text)
        #print(text_list)
        
        result = soup.find('div', class_="px-2 md:px-0")
        result2 = result.find_all('span', class_="font-small font-bold")
        #print(result2)
        #title_list=[] #小標（消費類型）
        for result in result2:
            title_list.append(result.text)
            #print(result.text+'\n')
        
        for i in range(len(title_list)):
            print(title_list[i])
        
        result = soup.find('div', class_="px-2 md:px-0")
        result2 = result.find_all('div', class_="flex-1 pl-2 text-right")
        #print(result2)
        text=[] #內文(現金回饋)
        
        for result in result2:
            text.append(result.text)
            #print(result.text+'\n')
            
        text.pop(0)
        for i in range(len(text)):
            print(text[i])
            text_list.append(text[i])
            
        card_dict1={
            "標題":title_list,
            "詳情":text_list
        }
        
        df = pd.DataFrame(card_dict1)
        folder_name = 'money101'+ranking_title[k+1]
        path = "/Users/takoyaki/Downloads/專題爬蟲/money101csv"  
    
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path, mode=0o777)
        path1 = str(path+'/'+folder_name+"/")
        #path='/Users/takoyaki/Downloads/專題爬蟲/money101csv/精選推薦卡csv/'
        df.to_csv(path1+"money101"+card_name[j]+"優惠.csv", index=False) #, index=False
        
    card_dict = {
        '卡片名稱': card_name[k+1],
        card_list2[0]:age_request,
        card_list2[1]:salary_request,
        card_list2[2]:fee,
        card_list2[3]:annual_fee,
        card_list2[4]:rate,
        card_list2[5]:agency
    }    
    
    
    df = pd.DataFrame(card_dict)
    folder_name = 'money101'+ranking_title[k+1]
    path = "/Users/takoyaki/Downloads/專題爬蟲/money101csv"  

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    #path1 = '/Users/takoyaki/Downloads/專題爬蟲/money101csv/精選推薦卡csv/' 
    df.to_csv(path0+"money101"+ranking_title[k+1]+"基本資訊"+'.csv', index=False)
    
driver.close() 

☆聯邦銀行 吉鶴卡
https://www.money101.com.tw/信用卡/產品/聯邦銀行吉鶴卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆第一銀行 iLEO卡
https://www.money101.com.tw/信用卡/產品/第一銀行ileo卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆滙豐銀行 匯鑽卡
https://www.money101.com.tw/信用卡/產品/滙豐銀行匯鑽卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆台新銀行 玫瑰Giving卡
https://www.money101.com.tw/信用卡/產品/台新銀行玫瑰giving卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆中國信託 LINE Pay卡
https://www.money101.com.tw/信用卡/產品/中國信託linepay卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆永豐銀行 MITSUI OUTLET PARK卡
https://www.money101.com.tw/信用卡/產品/永豐銀行mitsuioutletpark卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆永豐銀行 幣倍卡
https://www.money101.com.tw/信用卡/產品/永豐銀行幣倍卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆台新銀行 @GoGo iCash御璽卡
https://www.money101.com.tw/信用卡/產品/台新銀行@gogoicash御璽卡?entry=header&from=%E7%8F%BE%E9%87%91%E5%9B%9E%E9%A5%8B

☆滙豐銀行 現金回饋御璽卡
https://www.money101.com.tw/信用卡/產品/滙

聯邦銀行 吉鶴卡
最低年齡要求
國外刷卡手續費
循環信用利率
信用卡發卡機構
最低年齡要求:20歲

國外刷卡手續費:1.5 %

循環信用利率:循環利率：5%~15%；循環利率之基準日為104年9月1日（專案類卡別及預借現金商品依各專案約定，實際循環信用利率依申請時聯邦銀行信用卡網站公告為準）；預借現金手續費：預借金額x3.5%+NT$100（專案型預借現金商品手續費依各專案約定）；其他費用請至聯邦銀行卡友服務（各項費用計費說明）查詢。

信用卡發卡機構:JCB

產品特色
國內消費2%回饋無上限、日幣消費3.5%回饋無上限、保費1%回饋無上限

國內消費最高8%  活動期間新申辦「New New Bank」數位存款帳戶且綁定自動扣繳成功，享符合資格後至次月起算三個月內國內消費加碼6%回饋(活動期間總回饋上限1,000元)+國內消費2%回饋無上限 

日本消費最高5%  日本Apple Pay消費最高5%(含日本消費3.5%回饋，加碼1.5%月上限1,000元，需於日本提供QUIC Pay之場域使用Apple Pay感應支付) 日本亞馬遜、日本樂天市場兩大網購享最高5%回饋(含日本消費3.5%回饋，加碼1.5%月上限500元) 

國內日系服飾/生活/藥妝最高12%  指定名店：UNIQLO、大創、THREEPPY、Standard Products、台隆手創館、日藥本 舖、札幌藥妝、松本清、特美事、大葉高島屋、宜得利家居、唐吉軻德。 最高12%回饋=國內2%回饋無上限+新申辦New New Bank設定自動扣繳加碼6%(活動期間總回饋上限1,000元)+活動加碼4%(月上限500元) 

國內日系餐廳最高現折10%  勝博殿、花月嵐、一風堂、麵屋武藏、山頭火、吉豚屋、金子半之助、金大鋤壽喜 燒、欣葉日本料理、欣葉呷哺呷哺、日本橋海鮮丼、新宿內臟燒肉、本多日本料理，各現折優惠依吉鶴卡官網或餐廳公告為準。 

注意事項
定義  國內/外一般消費使用行動支付付款同樣享有現金回饋 一般消費回饋不包含： (1) 信用卡年費、延滯息、掛失費及依「聯邦信用卡約定條款」約定之各項手續費。 (2)	代扣各項事業費用或繳款、中華電信資費、學費、稅款、政府規費、交通罰款、車輛選/標號等交易。 (3)	特定賑災捐款、國外賭場取現交易、預借現金、專案型預借現金產品與餘額代償、證券代銷類、

台新銀行 @GoGo iCash御璽卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:不適用

國外刷卡手續費:1.5 %

年費:正卡年費：御璽卡NT$3000/免年費條件：核卡後申辦電子帳單或使用台新銀行帳戶(含Richart 帳戶)辦理自動轉帳代扣繳(含信用卡帳單、公用事業費用等)且不終止該服務，即享終生免年費；未辦理任一代扣繳服務，享首3年免年費，第4年起任刷1筆享當年度免年費。

循環信用利率:差別循環信用利率：一般消費及預借現金為6.75%~15%，依電腦評等而定；預借現金手續費：預借現金金額x3%，不足NT$100以NT$100計算；循環利率之基準日為104年9月1日；特店分期廣告已揭露各項相關費用總金額且無其他應繳費用，各期分期利率換算總費用年百分率為0%。

信用卡發卡機構:VISA

產品特色
精選網購、行動支付Go給力 最高3.8%  精選通路最高3.8%  基本回饋0.5%無上限，完成任務即升級享精選網購、行動支付最高3.8%回饋 精選網購：蝦皮、momo、PChome、Yahoo、Amazon 精選行動支付：LINE Pay、全支付、全盈+PAY、台新Pay     *精選消費最高3.8%包含〔(基本0.5%無上限)+(精選加碼3.3%，每期帳單回饋上限NT$1,000)〕    活動期間：2023/4/1~2024/3/31結帳帳單 

旅遊平安險刷＠GoGo卡支付80%以上團費，或全部公共交通工具費用，享NT$2,000萬旅遊平安險

注意事項
下載Richart、Richart Life APP (原台新卡得利)或行動銀行APP 享每筆消費推播通知

定義國內/外一般消費使用行動支付付款同樣享有現金回饋 一般消費不包含：歐盟地區實體商店交易、境外投資平台交易、特約商店分期付款交易及單筆消費分期0利率之交易、電子支付儲值卡自動加值、Samsung Pay悠遊卡加值、全聯福利中心消費、便利商店(惟綁定指定行動支付消費仍享回饋，其他支付方式則不回饋)、各項代扣繳費用、躉繳保費、投資型保費、三商美邦保費、彈性繳保費、藥安康藥局消費、臺北醫學大學附設醫院費用、各縣市路邊停車費、透過各繳費平台(包含但不限於線上支付/臨櫃繳款，如e-Bill全國繳費/i繳費/醫指付/電子化繳

第一銀行 鈦金商旅卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:固定年收入達新臺幣60萬元以上

國外刷卡手續費:1.5 %

年費:首年免年費、電子帳單免年費

循環信用利率:差別循環信用年利率為一銀放款基準利率+2.31%～9.31%｜差別循環信用年利率依電腦評定，最高為15%．利率之基準日為2015年9月1日｜預借現金手續費：預借金額X3.5%（最低新臺幣100元/3.5美元/400日圓/3歐元計）．其他收費項目及權益請上網查詢

信用卡發卡機構:Mastercard

產品特色
國內一般消費最高1%回饋 新臺幣累積一般消費帳單金額100,000元(含)以內：現金回饋0.6% 100,001元~1,000,000元(含)：現金回饋0.8% 1,000,001元以上：現金回饋1.0% 

外幣消費最高享4.4%現金回饋無上限  外幣消費3.2%現金回饋  活動期間：即日起~2023.12.31 海外外幣消費金額享3.2%現金回饋無上限。(含加碼0.7%現金回饋需登錄，回饋自登錄當月起算，恕不回溯)。  外幣消費月刷月有利  活動期間：即日起~2023.8.31 活動期間內刷鈦金商旅卡之外幣消費金額，當期消費滿額贈刷卡金，每期帳單回饋上限6,000元(回饋率最高1.2%)，每戶每期擇優回饋乙次。(需登錄)   

精選通路享最高3%現金回饋  活動期間：即日起~2023.12.31 精選通路：航空購票、免稅店、klook、kkday、台灣高鐵、台灣大車隊(55688 APP) 、大都會車隊、Uber APP、LINE TAXI、yoxi、汽車保養、購車訂金。      (活動需登錄，每戶每期帳單回饋上限500元)  

高鐵商務車廂限量升等優惠  活動期間：2023.1.1~2023.12.31 活動期間刷鈦金商旅卡上月帳單新增一般消費滿6萬元(正附卡合併計算)，當月享2張高鐵商務車廂免費升等。 

3,000萬元旅平險/1,000萬元海外全程意外險  保險期間：2023.4.1~2024.3.31 使用第一銀行鈦金商旅卡支付公共運輸工具票款之全部，或參加旅遊全部費用之80%(含)以上，將擁有「3,000萬元旅遊平安險」及30天的「1,000萬元海外全程意外險、100萬元海外意外傷害醫療

台新銀行 FlyGo卡
最低年齡要求
最低年薪要求
國外刷卡手續費
年費
循環信用利率
信用卡發卡機構
最低年齡要求:18 歲

最低年薪要求:不適用

國外刷卡手續費:1.5 %

年費:正卡NT$4500。免年費條件：使用台新行動帳單/電子帳單，即享免年費優惠。

循環信用利率:差別循環信用利率：一般消費及預借現金為6.75%~15%，依電腦評等而定；預借現金手續費：預借現金金額x3%，不足NT$100以NT$100計算；循環利率之基準日為104年9月1日；特店分期廣告已揭露各項相關費用總金額且無其他應繳費用，各期分期利率換算總費用年百分率為0%。

信用卡發卡機構:Mastercard

產品特色
精選航空、交通、訂房(票)刷出最高5%、海外最高3%!   基本回饋0.5%無上限，完成任務即升級享精選航空、交通、訂房(票)最高5%回饋！  精選航空: 華航、長榮、星宇、虎航 精選交通: 高鐵、中油直營 精選訂房(票): 雄獅、易遊網、可樂、Klook  海外(含網購) 3%無上限〔(基本0.5%無上限)+(海外加碼2.5%無上限)〕 其他一般消費 0.5% 活動期間：2023/4/1~2024/3/31結帳帳單 

卡友優惠 旅遊平安險：最高NT$3,000萬 海外旅遊全程保障保險：最高NT$1,000萬機場接送：機場接或送NT$428起/趟機場貴賓室優惠：龍騰貴賓室NT$299/次

注意事項
定義  國內/外一般消費使用行動支付付款同樣享有現金回饋 一般消費不包含：歐盟地區實體商店交易、境外投資平台交易、特約商店分期付款交易及單筆消費分期0利率之交易、電子支付儲值卡自動加值、Samsung Pay悠遊卡加值、全聯福利中心消、便利商店、各項代扣繳費用、躉繳保費、投資型保費、三商美邦保費、彈性繳保費、藥安康藥局消費、臺北醫學大學附設醫院費用、各縣市路邊停車費、透過各繳費平台繳納之各項費用、政府規費、違規罰鍰、透過「聯合信用卡處理中心」提供之信用卡小額支付特約商店之消費、預借現金及其手續費、預借現金分期金及其手續費、代償他行信用卡/現金卡/小額信貸等之金額、個人綜合所得稅、所得稅分期付款期金及其手續費、查核定稅、燃料費、違約金、循環利息、基金交易及其手續費、信用卡年費及一般信用卡相關手續費。  

產品特色
產品特色
注意事項
國內消費
國外消費
繳保費
加油消

AttributeError: 'NoneType' object has no attribute 'find_all'

In [75]:
#print(len(age_request))
#print(len(salary_request))
#print(len(fee))
#print(len(annual_fee))
#print(len(rate))
#print(len(agency))

129
129
129
128
127
124
